In [ ]:
# 🧪 Python Kodēšana 2 – Tests (1.–4. nodarbība) - Timer Cell
from IPython.display import display, clear_output
import time
from datetime import datetime
import requests
import ipywidgets as widgets

# Global state
submitted = False
results = [None] * 8
attempts = [0] * 8

# Google Forms configuration
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdTSrnEP9GQajJRRCKqQMbwWGS_TL3o3B4T_fequkWmUZQCCg/formResponse"
entry_ids = {
    "Students": "entry.1649720525",
    "Datums": "entry.667959859",
    "Rezultats": "entry.940605140",
    "1": "entry.213159259",
    "2": "entry.267050949",
    "3": "entry.418893037",
    "4": "entry.1447996441",
    "5": "entry.1914428828",
    "6": "entry.1728217231",
    "7": "entry.1875638404",
    "8": "entry.907839017"
}

# Shared state
name_input = widgets.Text(description='Vārds:', placeholder='Ievadi savu vārdu')
questions_data = [
    ("Ko dara pygame.init()?", [
        "Inicializē visus importētos pygame moduļus",
        "Palaiž spēles ciklu",
        "Ielādē spēles logu",
        "Izveido galveno varoņa sprite objektu"
    ], 0),
    ("Kura metode tiek izmantota, lai uzzīmētu taisnstūri pygame logā?", [
        "pygame.draw.rect()",
        "pygame.screen.rectangle()",
        "pygame.create.rect()",
        "pygame.render.rect()"
    ], 0),
    ("Kā tiek atjaunināts logs pygame pēc zīmēšanas?", [
        "pygame.display.flip()",
        "pygame.update.display()",
        "pygame.screen.refresh()",
        "pygame.render.frame()"
    ], 0),
    ("Ko pārbauda event.type == pygame.QUIT?", [
        "Vai lietotājs ir nospiedis loga aizvēršanas pogu",
        "Vai lietotājs nospieda taustiņu Enter",
        "Vai spēlei jāsākas no jauna",
        "Vai spēlētājs zaudēja"
    ], 0),
    ("Kurš modulis tiek izmantots, lai veidotu GUI lietotnes Python valodā?", [
        "tkinter",
        "pygame.gui",
        "guimodule",
        "tkbuilder"
    ], 0),
    ("Kurš tkinter logelements tiek izmantots, lai saņemtu tekstu no lietotāja?", [
        "Entry",
        "Label",
        "Button",
        "Canvas"
    ], 0),
    ("Ko dara mainloop() tkinter lietotnē?", [
        "Palaiž GUI notikumu ciklu",
        "Pieslēdzas datubāzei",
        "Atjauno Python vidi",
        "Izveido loga izvietojumu"
    ], 0),
    ("Kurš tkinter logelements ļauj zīmēt figūras kā taisnstūrus un tekstu?", [
        "Canvas",
        "Frame",
        "Box",
        "Entry"
    ], 0)
]

# Shuffle questions
import random
random.shuffle(questions_data)

# Timer display
refresh_button = widgets.Button(description="Atjaunot taimeri", button_style='warning')

def refresh_timer(_):
    clear_output(wait=True)
    print(f"Atlikušais laiks: {timer_text}")
    display(refresh_button)

refresh_button.on_click(refresh_timer)

# Submission function
def send_to_google_form(_):
    global submitted
    if submitted:
        return
    submitted = True
    student_name = name_input.value.strip() or "Nezināms"
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    form_data = {
        entry_ids["Students"]: student_name,
        entry_ids["Datums"]: now,
        entry_ids["Rezultats"]: str(final_score) + "%"
    }
    for i, (q, _, _) in enumerate(questions_data):
        question_entry = entry_ids[str(i+1)]
        response = f"{q} (Mēģinājumi: {attempts[i]}) – {results[i] if results[i] is not None else 0} punkti"
        form_data[question_entry] = response
    try:
        response = requests.post(form_url, data=form_data, timeout=10)
        if response.status_code == 200:
            print("Rezultāti veiksmīgi nosūtīti uz Google Formu!")
        else:
            print(f"Kļūda sūtot datus (statuss: {response.status_code})")
    except Exception as e:
        print(f"Kļūda sūtot datus: {e}")

# Simplified timer loop
timer_text = "40:00"
minutes = 40
seconds = minutes * 60
start_time = time.time()
max_runtime = seconds + 15  # Prevent infinite loop
try:
    while seconds > 0 and not submitted:
        elapsed = time.time() - start_time
        if elapsed > max_runtime:
            print("Timer stalled. Please restart runtime.")
            break
        remaining = max(0, seconds - int(elapsed))
        mins, secs = divmod(remaining, 60)
        timer_text = f"{mins:02d}:{secs:02d}"
        clear_output(wait=True)
        print(f"Atlikušais laiks: {timer_text}")
        display(refresh_button)
        time.sleep(1)
    if not submitted:
        timer_text = "Laiks beidzies! Rezultāti nosūtīti."
        clear_output(wait=True)
        print(timer_text)
        send_to_google_form(None)
except Exception as e:
    print(f"Timer error: {e}")


In [ ]:
# 🧪 Python Kodēšana 2 – Tests (1.–4. nodarbība) - Text Quiz Cell
from IPython.display import display, clear_output
import random
import sys

# Use shared state (submitted, results, attempts, questions_data, name_input, form_url, entry_ids)
def run_quiz():
    try:
        for i, (question, options, correct_index) in enumerate(questions_data):
            if results[i] is not None or submitted:
                continue
            paired = list(enumerate(options))
            random.shuffle(paired)
            shuffled_indexes, shuffled_options = zip(*paired)
            correct_shuffled_index = shuffled_indexes.index(correct_index)
            clear_output(wait=True)
            print(f"{i+1}. {question}")
            for j, opt in enumerate(shuffled_options):
                print(f"  {j+1}. {opt}")
            start_time = time.time()
            while results[i] is None and not submitted:
                if time.time() - start_time > 60:  # Timeout per question
                    print("Input timeout. Moving to next question.")
                    break
                try:
                    answer = input("Ievadi atbildes numuru (1-4): ").strip()
                    if not answer:
                        print("Ievadi skaitli.")
                        continue
                    answer = int(answer) - 1
                    if 0 <= answer < 4:
                        attempts[i] += 1
                        if answer == correct_shuffled_index:
                            penalty = 5 * (attempts[i] - 1)
                            score = max(12.5 - penalty, 0)
                            results[i] = score
                            print("✔️ Pareizi!")
                            time.sleep(1)
                            break
                        else:
                            print("❌ Nepareizi. Mēģini vēlreiz.")
                    else:
                        print("Nederīgs numurs. Ievadi 1-4.")
                except ValueError:
                    print("Ievadi skaitli.")
                except (EOFError, KeyboardInterrupt):
                    print("Input interrupted. Try again.")
                time.sleep(1)
                clear_output(wait=True)
                print(f"{i+1}. {question}")
                for j, opt in enumerate(shuffled_options):
                    print(f"  {j+1}. {opt}")

        if all(r is not None for r in results) and not submitted:
            clear_output(wait=True)
            total = sum(r if r else 0 for r in results)
            final_score = round(total)
            status = "✅ Nokārtots!" if final_score >= 70 else "❌ Nenokārtots."
            print(f"Rezultāts: {final_score}% – {status}")
            display(name_input)
            submit_button = widgets.Button(description="Sūtīt uz Google Formu", button_style='primary')
            submit_button.on_click(send_to_google_form)
            display(submit_button)
    except Exception as e:
        print(f"Quiz error: {e}")
        sys.exit(1)

run_quiz()
